### VLM model

In [10]:
from datasets import load_dataset
cppe5 = load_dataset("cppe-5")

INFO:datasets:PyTorch version 2.3.0 available.
INFO:datasets:TensorFlow version 2.10.0 available.


In [5]:
from transformers import AutoImageProcessor

checkpoint = "facebook/detr-resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## Preprocessing 

1) resizing the image
2) Darken or Lighten the image(Based on probability)
3) Anything Else to Add more noises to the the data

The field are all copy pasted as of now, chage it according to what is needed


In [12]:
import albumentations
import numpy as np
import torch

transform = albumentations.Compose(
    [
        albumentations.Resize(480, 480),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.Blur(blur_limit=7, always_apply=True, p=1),
        albumentations.RandomBrightnessContrast(p=1),
    ],
    bbox_params=albumentations.BboxParams(format="coco", label_fields=["category"]),
)

In [6]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):
        new_ann = {
            "image_id": image_id,
            "category_id": category[i],
            "isCrowd": 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }
        annotations.append(new_ann)

    return annotations

In [7]:
# transforming a batch
def transform_aug_ann(examples):
    
    image_ids = examples["image_id"]
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples["image"], examples["objects"]):
        image = np.array(image.convert("RGB"))[:, :, ::-1]
        out = transform(image=image, bboxes=objects["bbox"], category=objects["category"])

        area.append(objects["area"])
        images.append(out["image"])
        bboxes.append(out["bboxes"])
        categories.append(out["category"])

    targets = [
        {"image_id": id_, "annotations": formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]

    return image_processor(images=images, annotations=targets, return_tensors="pt")

In [11]:
cppe5["train"] = cppe5["train"].with_transform(transform_aug_ann)